In [36]:
import importlib
import sys
from tqdm import tqdm
import  os
import glob
import rawEventBuilder
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "Osiris//monitoring//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import mplhep as hep
import Timing_tools as TTools
import rawFileReader
import Visual_tools as VTools
import datetime

hep.style.use([hep.style.ATLAS])

# Specify the directory
data_list = sorted([f for f in os.listdir("data//reset") if os.path.isfile(os.path.join("data//reset", f))], reverse=True) ##all files in data directory sorted from the newest to the oldest
print(data_list)
file_path = [dir_path+"//data//"+"proAnubis_240731_0217.raw"] #
file_path = list(map(lambda p: dir_path+"data//reset//"+p, data_list))[-1:] # insert your file
print(file_path)

['proAnubis_240810_2240.raw', 'proAnubis_240810_2040.raw', 'proAnubis_240810_1840.raw', 'proAnubis_240810_1640.raw', 'proAnubis_240810_1440.raw', 'proAnubis_240810_1240.raw', 'proAnubis_240810_1040.raw', 'proAnubis_240810_0840.raw', 'proAnubis_240810_0640.raw', 'proAnubis_240810_0440.raw', 'proAnubis_240810_0240.raw', 'proAnubis_240810_0040.raw']
['C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_1440.raw']


In [17]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
importlib.reload(VTools)

fReader = rawFileReader.fileReader(file_path[0]) # load in the classs object
#fReader.skip_events(10_000_000)
initial_event_chunk = fReader.get_aligned_events(interval=100)
print(RTools.abs_badVsgood_hits_count(initial_event_chunk))

#VTools.time_events(initial_event_chunk)
#VTools.hitHeatMap(initial_event_chunk,0)

[[298, 46], [271, 47], [274, 14], [246, 42], [144, 55]]


In [38]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
importlib.reload(VTools)

import matplotlib.pyplot as plt
interval = 100 # Set your monitoring chunck size
order = [(0,1), (1,2), (2,3), (3,4)] # Order what you want to align
actual_mets = {}
last_reset = None
for file in file_path:
    max_process_event_chunk = 4_000 # End the loop early
    file_name = file.split("//")[-1][:-4]
    fReader = rawFileReader.fileReader(file) # reload in the classs object
    processedEvents = 0 # Initialisation

    #Initialise variables to store the results
    mets = [[] for pair in range(len(order))]
    tdc_event_count = [[[],[]] for tdc in range(5)]# prepare for histogram plotting
    initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval) # get the initial event chunk
    #originTime = max([event_chunk[0].tdcEvents[tdc].time for tdc in range(5) if event_chunk[0].tdcEvents[tdc].time])
    #print(originTime)
    binsx = []
    max_chunk_duration = datetime.timedelta(0)
    with tqdm(total=max_process_event_chunk, desc=f"Processing {file_name}", unit='Events') as pbar:
        while processedEvents < max_process_event_chunk:
            processedEvents += 1
            try:
                event_chunk = fReader.get_aligned_events(order=order, interval=interval) # get the aligned events
                if not event_chunk:
                    #print("Misaligned:", processedEvents)
                    for i, (tdc1, tdc2) in enumerate(order):
                        mets[i].append(0) #I assume it is a zero but might not be right
                    continue
            except Exception as e:
                print("Exception:", e)
                max_process_event_chunk = processedEvents
                break
            
            event_time = max([event_chunk[0].tdcEvents[tdc].time for tdc in range(5) if event_chunk[0].tdcEvents[tdc].time])
            
            if last_reset:
                if (event_time - last_reset).total_seconds() > 60:
                    last_reset = event_time
                    fReader.evtBuilder = rawEventBuilder.eventBuilder()
            else:
                last_reset = event_time
                originTime = event_time
            
            for i, (tdc1, tdc2) in enumerate(order):
                mets[i].append(VTools.metric_possible(event_chunk, tdc1, tdc2)[0])
            tdc_event_count_buffer = RTools.abs_badVsgood_hits(event_chunk) # finding the number o
            
            last_event_time = max([event_chunk[-1].tdcEvents[tdc].time for tdc in range(5) if event_chunk[-1].tdcEvents[tdc].time])
            first_event_time = max([event_chunk[0].tdcEvents[tdc].time for tdc in range(5) if event_chunk[0].tdcEvents[tdc].time])
           # print("First event time:", first_event_time )
           # print(first_event_time - originTime)
            #print((first_event_time - originTime).total_seconds())
            if binsx:
                binsx.append((first_event_time - originTime).total_seconds())
            else:
                originTime = first_event_time
                binsx.append((first_event_time - originTime).total_seconds()) # stupid ik
            if last_event_time - first_event_time > max_chunk_duration:
                max_chunk_duration = last_event_time - first_event_time
            for tdc in  range(5):
                tdc_event_count[tdc][0].append(tdc_event_count_buffer[tdc][0])
                tdc_event_count[tdc][1].append(tdc_event_count_buffer[tdc][1])
            
            pbar.update(1)
    """
    print(f"Max Chunk Duration: {max_chunk_duration}")
    binsx = sorted(binsx)
    actual_mets[file_name] = tdc_event_count
    fig, ax = plt.subplots(figsize=(10, 8))
    for tdc in range(0,5):
        met = tdc_event_count[tdc]
        ax.plot(binsx, met[0], label=f'Good hits TDC{tdc}')
        ax.plot(binsx, met[1], label=f'Bad hits TDC{tdc}')

    #ax.set_xlim(0, max_process_event_chunk)
    # ax.set_ylim(-1, 100)
    ax.legend()
    ax.set_title('Abs Good vs Bad')
    ax.set_ylabel('num of events')
    ax.set_xlabel('Absolute time / s')
    plt.savefig(f"hits_real_time_{file_name}.png")
    """
    fig, ax = plt.subplots(figsize=(10, 8))
    x_axis = [i for i in range(max_process_event_chunk)]
    for i, pair in enumerate(order):
        ax.plot( x_axis, mets[i], label=f'TDC{pair[0]} vs TDC{pair[1]}')
    ax.legend()
    ax.set_title('Possible alignement data count')
    ax.set_ylabel('num of events')
    ax.set_xlabel('Event Chunk number')
    plt.savefig(f"possible_{file_name}.png")
    
    

Processing proAnubis_240810_1440:  52%|█████▏    | 2098/4000 [01:04<00:35, 53.07Events/s]

In [24]:
print(f"Max Chunk Duration: {max_chunk_duration}")
print("binsx", binsx)
binsx = sorted(binsx)
actual_mets[file_name] = tdc_event_count
fig, ax = plt.subplots(figsize=(10, 8))
for tdc in range(0,5):
    met = tdc_event_count[tdc]
    ax.plot(binsx, met[0], label=f'Good hits TDC{tdc}')
    ax.plot(binsx, met[1], label=f'Bad hits TDC{tdc}')

#ax.set_xlim(0, max_process_event_chunk)
# ax.set_ylim(-1, 100)
ax.legend()
ax.set_title('Abs Good vs Bad')
ax.set_ylabel('num of events')
ax.set_xlabel('Absolute time / s')
plt.savefig(f"hits_real_time_{file_name}.png")